In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
DEFAULT_CRS = "EPSG:4326"

In [3]:

hex_sp = gpd.read_file("data/shapeFiles/GRIDS_H3_SP_RES10") 


# Transform the CRS to EPSG:31983
hex_sp = hex_sp.to_crs(DEFAULT_CRS)
hex_sp.head()

,id_hex,geometry
0,8aa8100c3257fff,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5..."
1,8aa8100d2447fff,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5..."
2,8aa8100e1637fff,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5..."
3,8aa8100f0827fff,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6..."
4,8aa81010ec07fff,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8..."


# IPTU Lotes

In [4]:
iptu_lotes = gpd.read_parquet('data/iptu_por_lotes.parquet')

In [5]:
iptu_lotes.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
iptu_lotes.head()

,sqlc,sq,agregado,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos lotes,Área Total Ocupada,...,Residencial horizontal Alto (qt),Comercial vertical Baixo (qt),Comercial vertical Médio (qt),Comercial vertical Alto (qt),Comercial horizontal Baixo (qt),Comercial horizontal Alto (qt),Comercial horizontal Médio (qt),Terreno (qt),Outros Usos (qt),geometry
0,145146007500,145146,False,2024.0,1,0,NaN,94.0,94.0,45.0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((348867.993 7392147.203, 348868.685 7..."
1,145110006000,145110,False,2024.0,1,0,NaN,145.0,145.0,69.0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((349753.035 7392543.100, 349753.120 7..."
2,145110005300,145110,False,2024.0,1,0,NaN,287.0,287.0,0.0,...,0,0,0,0,0,0,0,1,0,"POLYGON ((349753.035 7392543.100, 349753.043 7..."
3,145148003500,145148,False,2024.0,1,0,NaN,94.0,94.0,34.0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((348883.423 7391901.839, 348887.269 7..."
4,145157009800,145157,False,2024.0,1,0,NaN,125.0,125.0,35.0,...,0,0,0,0,0,0,0,0,0,"POLYGON ((348732.931 7391929.804, 348733.063 7..."


In [19]:
iptu_lotes_final = iptu_lotes[['sqlc', 'Quantidade de Unidades', 'geometry']]

In [20]:
iptu_lotes_final['qnt_unidades_m2'] = iptu_lotes_final['Quantidade de Unidades'] / (iptu_lotes_final.to_crs("EPSG:5641").area)

/opt/anaconda3/envs/scriptsMestrado/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [21]:
iptu_lotes_final.head()

,sqlc,Quantidade de Unidades,geometry,qnt_unidades_m2
0,145146007500,1,"POLYGON ((-46.48095 -23.57407, -46.48094 -23.5...",0.008894
1,145110006000,1,"POLYGON ((-46.47224 -23.57058, -46.47224 -23.5...",0.005801
2,145110005300,1,"POLYGON ((-46.47224 -23.57058, -46.47224 -23.5...",0.002841
3,145148003500,1,"POLYGON ((-46.48082 -23.57629, -46.48078 -23.5...",0.008024
4,145157009800,1,"POLYGON ((-46.48229 -23.57602, -46.48229 -23.5...",0.006843


# Intersecção

In [22]:
if iptu_lotes_final.crs != DEFAULT_CRS:
    iptu_lotes_final = iptu_lotes_final.to_crs(DEFAULT_CRS)
if hex_sp.crs != DEFAULT_CRS:
    hex_sp = hex_sp.to_crs(DEFAULT_CRS)

In [23]:
iptu_lotes_join_h3 =  iptu_lotes_final.overlay(hex_sp[['id_hex', 'geometry']], how='intersection')


/opt/anaconda3/envs/scriptsMestrado/lib/python3.12/site-packages/geopandas/geodataframe.py:2475: UserWarning: `keep_geom_type=True` in overlay resulted in 2 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [24]:
iptu_lotes_join_h3.head()

,sqlc,Quantidade de Unidades,qnt_unidades_m2,id_hex,geometry
0,145146007500,1,0.008894,8aa8100990effff,"POLYGON ((-46.48094 -23.57406, -46.48093 -23.5..."
1,145110006000,1,0.005801,8aa810099797fff,"POLYGON ((-46.47224 -23.57058, -46.47223 -23.5..."
2,145110005300,1,0.002841,8aa810099797fff,"POLYGON ((-46.47224 -23.57058, -46.47222 -23.5..."
3,145148003500,1,0.008024,8aa81009900ffff,"POLYGON ((-46.48078 -23.57625, -46.48063 -23.5..."
4,145157009800,1,0.006843,8aa810099067fff,"POLYGON ((-46.48254 -23.57607, -46.48254 -23.5..."


In [25]:
iptu_lotes_join_h3[iptu_lotes_join_h3['sqlc'] == '005076017300']

,sqlc,Quantidade de Unidades,qnt_unidades_m2,id_hex,geometry
2266848,005076017300,1,0.001174,8aa8100c0747fff,"POLYGON ((-46.62905 -23.55272, -46.62906 -23.5..."
2266849,005076017300,1,0.001174,8aa8100c074ffff,"POLYGON ((-46.62910 -23.55261, -46.62916 -23.5..."
2266850,005076017300,1,0.001174,8aa8100c075ffff,"POLYGON ((-46.62898 -23.55257, -46.62897 -23.5..."


## Krigatem simples e Agrupar 

In [26]:
iptu_lotes_join_h3['qnt_unidades']  = iptu_lotes_join_h3['qnt_unidades_m2'] * (iptu_lotes_join_h3.to_crs("EPSG:5641").area)

In [27]:
iptu_lotes_join_h3.head()

,sqlc,Quantidade de Unidades,qnt_unidades_m2,id_hex,geometry,qnt_unidades
0,145146007500,1,0.008894,8aa8100990effff,"POLYGON ((-46.48094 -23.57406, -46.48093 -23.5...",1.000000
1,145110006000,1,0.005801,8aa810099797fff,"POLYGON ((-46.47224 -23.57058, -46.47223 -23.5...",1.000000
2,145110005300,1,0.002841,8aa810099797fff,"POLYGON ((-46.47224 -23.57058, -46.47222 -23.5...",1.000000
3,145148003500,1,0.008024,8aa81009900ffff,"POLYGON ((-46.48078 -23.57625, -46.48063 -23.5...",1.000000
4,145157009800,1,0.006843,8aa810099067fff,"POLYGON ((-46.48254 -23.57607, -46.48254 -23.5...",0.410508


In [30]:
iptu_lotes_agg = iptu_lotes_join_h3.groupby(['id_hex']).agg({
    "qnt_unidades": ['sum', 'mean', 'median', 'std', 'var'],
}).reset_index()

In [31]:
iptu_lotes_agg.columns = ['_'.join(col).strip() for col in iptu_lotes_agg.columns.values if col != 'id_hex']


In [32]:
iptu_lotes_agg.head()

,id_hex_,qnt_unidades_sum,qnt_unidades_mean,qnt_unidades_median,qnt_unidades_std,qnt_unidades_var
0,8aa810000007fff,16.349822,0.653993,0.782407,0.378914,0.143576
1,8aa81000000ffff,3.705577,0.336871,0.289419,0.293076,0.085894
2,8aa810000017fff,8.211257,0.513204,0.528788,0.407606,0.166142
3,8aa81000001ffff,6.262170,0.521848,0.622296,0.383182,0.146828
4,8aa810000027fff,14.249014,0.647682,0.936345,0.419149,0.175686


In [34]:
iptu_lotes_agg[iptu_lotes_agg['id_hex_'] == '8aa8100c0747fff']

,id_hex_,qnt_unidades_sum,qnt_unidades_mean,qnt_unidades_median,qnt_unidades_std,qnt_unidades_var
20099,8aa8100c0747fff,1544.10458,48.253268,23.617943,58.845241,3462.762426


In [35]:
hex_iptu = hex_sp.join(iptu_lotes_agg.set_index('id_hex_'), how='left', on='id_hex')

In [36]:
hex_iptu.head()

,id_hex,geometry,qnt_unidades_sum,qnt_unidades_mean,qnt_unidades_median,qnt_unidades_std,qnt_unidades_var
0,8aa8100c3257fff,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5...",187.921324,3.545685,1.000000,6.992005,48.888136
1,8aa8100d2447fff,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5...",110.909650,1.440385,0.850019,6.770837,45.844233
2,8aa8100e1637fff,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5...",99.565528,5.531418,0.974683,14.344737,205.771485
3,8aa8100f0827fff,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6...",0.019248,0.019248,0.019248,NaN,NaN
4,8aa81010ec07fff,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8...",NaN,NaN,NaN,NaN,NaN


In [37]:
hex_iptu.fillna(0, inplace=True)

In [38]:
hex_iptu.isna().sum()

id_hex                 0
geometry               0
qnt_unidades_sum       0
qnt_unidades_mean      0
qnt_unidades_median    0
qnt_unidades_std       0
qnt_unidades_var       0
dtype: int64

In [39]:

# Write the GeoDataFrame to a shapefile
hex_iptu.to_file("data/WGS84_hex_iptu.gpkg", driver="GPKG")
